In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'  # Suppress TensorFlow GPU warnings

In [2]:
import cv2
import numpy as np

In [3]:
from keras.applications import MobileNetV2
from keras.layers import Dropout, Dense, GlobalAveragePooling2D
from tensorflow.keras.utils import to_categorical
from keras.models import Model

In [4]:
## things required for training and testing
image_size = 224
label_lst = ['A', 'Aa', 'Ah', 'Ai', 'Am', 'Au', 'Ba', 'Bha', 'Ca', 'Cha', 'Da', 'Dha', 'D_a', 'D_ha', 'E', 'Ee', 'Ee_', 'E_', 'Ga', 'Gha', 'Ha', 'I', 'Ii', 'Ilh', 'Ill', 'In', 'Inh', 'Irr', 'Ja', 'Ka', 'Kha', 'La', 'Lha', 'Ma', 'Na', 'Nga', 'Nha', 'Nothing', 'N_a', 'O', 'Oo', 'Pa', 'Pha', 'R', 'Ra', 'Rha', 'Sa', 'Sha', 'Shha', 'Space', 'Ta', 'Tha', 'T_a', 'T_ha', 'U', 'Uu', 'Uu_', 'U_', 'Va', 'Ya', 'Zha']
NUM_CLASSES = len(label_lst)

In [23]:
## things required for training

dataset_paths = ['train_data_combined']#, 'train_data_2', 'train_data_1']

def load_dataset(dataset_paths, image_size):
    dataset = []
    for dataset_path in dataset_paths:
        for directory in os.listdir(dataset_path):
            path = os.path.join(dataset_path, directory)
            if not os.path.isdir(path):
                continue
            for item in os.listdir(path):
                if item.startswith("."):
                    continue
                if not item.endswith(".jpg"):
                    continue
                img = cv2.imread(os.path.join(path, item))
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img = cv2.resize(img, (image_size, image_size))
                dataset.append([img, directory])
            print(f"{directory}", end=" ")
    return dataset

CLASS_MAP = {label_lst[i]: i for i in range(NUM_CLASSES)}
def mapper_for_train(val):
    return CLASS_MAP[val]

In [24]:
dataset = load_dataset(dataset_paths, image_size)

A Aa Ah Ai Am Au Ba Bha Ca Cha Da Dha D_a D_ha E Ee Ee_ E_ Ga Gha Ha I Ii Ilh Ill In Inh Irr Ja Ka Kha La Lha Ma Na Nga Nha Nothing N_a O Oo Pa Pha R Ra Rha Sa Sha Shha Space Ta Tha T_a T_ha U Uu Uu_ U_ Va Ya Zha 

In [25]:
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(image_size, image_size, 3))
# Freeze convolutional layers of MobileNetV2
for layer in base_model.layers:
    layer.trainable = False
# Add custom top layers for classification
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
x = Dense(512, activation='relu')(x)  # Add additional dense layer for better feature representation
predictions = Dense(NUM_CLASSES, activation='softmax')(x)
# Create the model
model = Model(inputs=base_model.input, outputs=predictions)
# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [26]:
##things required for testing

def load_alphabet_images(folder, image_size):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder, filename))
        img = cv2.resize(img, (image_size, image_size))  # Resize image to a uniform size
        images.append(img)
    return np.array(images)

REV_CLASS_MAP = {i:label_lst[i] for i in range(NUM_CLASSES)}

def mapper_for_test(val):
    return REV_CLASS_MAP[val]

testset_paths = ['test_data_combined', 'test_data_1', 'test_data_2']

In [27]:
#train model

x=60 #max=60
current_dataset1 = dataset[:x*200+200]
current_dataset2 = dataset[200*61:200*61+x*200+200]
current_dataset = current_dataset1 + current_dataset2
for i in current_dataset[::200]:
    print(i[1],end=" ")

data, labels = zip(*current_dataset)
labels = list(map(mapper_for_train, labels))
labels = to_categorical(labels, num_classes=NUM_CLASSES)

model.fit(np.array(data), np.array(labels), epochs=7)

A A Aa Aa Ah Ah Ai Ai Am Am Au Au Ba Ba Bha Bha Ca Ca Cha Cha Da Da Dha Dha D_a D_a D_ha D_ha E E Ee Ee Ee_ Ee_ E_ E_ Ga Ga Gha Gha Ha Ha I I Ii Ii Ilh Ilh Ill Ill In In Inh Inh Irr Irr Ja Ja Ka Ka Kha Kha La La Lha Lha Ma Ma Na Na Nga Nga Nha Nha Nothing Nothing N_a N_a O O Oo Oo Pa Pa Pha Pha R R Ra Ra Rha Rha Sa Sa Sha Sha Shha Shha Space Space Ta Ta Tha Tha T_a T_a T_ha T_ha U U Uu Uu Uu_ Uu_ U_ U_ Va Va Ya Ya Zha Zha Epoch 1/7
763/763 ━━━━━━━━━━━━━━━━━━━━ 517s 661ms/step - accuracy: 0.4569 - loss: 2.1611
Epoch 2/7
763/763 ━━━━━━━━━━━━━━━━━━━━ 515s 674ms/step - accuracy: 0.8875 - loss: 0.3768
Epoch 3/7
763/763 ━━━━━━━━━━━━━━━━━━━━ 519s 679ms/step - accuracy: 0.9204 - loss: 0.2521
Epoch 4/7
763/763 ━━━━━━━━━━━━━━━━━━━━ 524s 686ms/step - accuracy: 0.9324 - loss: 0.2058
Epoch 5/7
763/763 ━━━━━━━━━━━━━━━━━━━━ 509s 667ms/step - accuracy: 0.9430 - loss: 0.1741
Epoch 6/7
763/763 ━━━━━━━━━━━━━━━━━━━━ 487s 638ms/step - accuracy: 0.9435 - loss: 0.1685
Epoch 7/7
763/763 ━━━━━━━━━━━━━━━━━━━━ 4

In [28]:
##test model

test_data_folder = testset_paths[0]

correct_predictions = 0
total_predictions = 0

for alphabet_folder in sorted(os.listdir(test_data_folder))[:x+1]:
    path = os.path.join(test_data_folder,alphabet_folder)
    if not os.path.isdir(path):
        continue
    for file_name in os.listdir(path):
        if file_name.startswith(".") and not file_name.endswith(".jpg"):
            continue
            
        # prepare the image
        image_path = os.path.join(path,file_name)
        img = cv2.imread(image_path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (image_size, image_size))

        # predict the sign made
        pred = model.predict(np.array([img]))
        sign_code = np.argmax(pred[0])
        sign_name = mapper_for_test(sign_code)

        ground_truth = alphabet_folder

        # Compare predicted label with ground truth label
        if sign_name == ground_truth:
            correct_predictions += 1

        total_predictions += 1

        print(f"Predicted: {sign_name} for image {image_path}")
        
accuracy = correct_predictions / total_predictions
print(f"\n\nAccuracy: {accuracy * 100:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
Predicted: Ya for image test_data_combined\A\1.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 94ms/step
Predicted: A for image test_data_combined\A\10.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step
Predicted: A for image test_data_combined\A\11.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted: A for image test_data_combined\A\12.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Predicted: A for image test_data_combined\A\13.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step
Predicted: A for image test_data_combined\A\14.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step
Predicted: A for image test_data_combined\A\15.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 64ms/step
Predicted: A for image test_data_combined\A\16.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step
Predicted: A for image test_data_combined\A\17.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step
Predicted: A for image test_data_combined\A\18.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step
Predicted: A for image test_data_combined\A\19.jpg
1/1 ━━━━━━━━━━━━━━━━━━━

In [29]:
model.save('train_data_combined_acc_.keras')